# Benchmarking of Optimization Algorithms

This IJulia notebook goes through the steps to benchmark various optimization algorithms using random tensor networks. [Results](#report)


In [2]:
] add https://github.com/Qaintum/Qaintessent.jl#view-matrix-gate

   Updating git-repo `https://github.com/Qaintum/Qaintessent.jl`


   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
   Updating `/import/home/ga53vuw/Documents/PhD/projects/QAI/Qaintensor.jl/Project.toml`
 [no changes]
   Updating `/import/home/ga53vuw/Documents/PhD/projects/QAI/Qaintensor.jl/Manifest.toml`
 [no changes]


In [3]:
using Test
using TestSetExtensions 
using Qaintessent
using Qaintensor
using BenchmarkTools
using LinearAlgebra
using StatsBase: sample
using TensorOperations

## Setup
    
Defining helper functions to create random unitary gates

In [4]:
function shift_summation(S::Summation, step::Integer)
   return Summation([S.idx[i].first + step => S.idx[i].second for i in 1:2])
end

function shift_pair(P::Pair{Integer, Integer}, step::Integer)
    return P.first + step => P.second
end

function rand_U(M)
    U, _ = qr(rand(ComplexF64, 2^M, 2^M))
    @assert U*adjoint(U) ≈ I
    return Array(U)
end

# Random unitary matrix in M
function rand_local_g(M, N, max_d)
    @assert M ≤ max_d ≤ N
    U = rand_U(M)
    dist = rand()
    iwire = sort(sample(1:max_d, M, replace = false))
    m1 = rand(0:N-maximum(iwire))
    iwire .+= m1

    return CircuitGate{M, N, AbstractGate{M}}(Tuple(iwire), MatrixGate(U))
end

rand_local_g (generic function with 1 method)

## Case 1: Log Depth Tensor Network

Define `log_depth_TN`. This function creates a random logarithmic depth tensor network composed of random unitary gates. Note that the ends of this randomly generated tensor networks are 1-Tensors

In [5]:
function log_depth_TN(M, N, C)
    Nlayers = C*Int(round(log2(N)))
    gates = AbstractCircuitGate{N}[]
    for j in 1:Nlayers
        itergates = (j%2 == 1) ? (1:M:N-M+1) : (M:M:N-M+1)
        for i in itergates
            U = rand_U(M)
            g = CircuitGate{M, N, AbstractGate{M}}(Tuple(collect(i:i+M-1)), MatrixGate(U))
            push!(gates, g)
        end
    end
    cgc = CircuitGateChain{N}(gates)
    # make tensor network

    tensors = Tensor.([[1,0] for i in 1:N])
    contractions = Summation[]
    openidx = [i => 1 for i in 1:N]
    T = TensorNetwork(tensors, contractions, openidx)

    tensor_circuit!(T, cgc)
    # measure
    for i in 1:N
        push!(T.tensors, deepcopy(T.tensors[N+1-i]))
        push!(T.contractions, Summation([T.openidx[end], length(T.tensors) => 1]))
        pop!(T.openidx)
    end
    T, cgc
end


log_depth_TN (generic function with 1 method)

In [6]:
printstyled("\nRunning contraction optimization for log depth TN circuit\n"; color=:green)
for N in 2:4
    Ngates = 10
    M = 2
    max_d = 3
    ψ, cgc = log_depth_TN(M, N, 1);

    printstyled("\nBaseline contraction\n"; color=:yellow)
    ref = contract(ψ)
    printstyled("opt_einsum contraction\n"; color=:yellow)
    einsum_ans = contract(ψ; optimize=true)
    printstyled("treewidth contraction\n"; color=:yellow)
    printstyled("  time for treewidth optimization: "; color=:blue)
    @time optimize_contraction_order!(ψ)
    opt_ans = contract(ψ)

    @test ref ≈ einsum_ans
    @test ref ≈ opt_ans
end



Running contraction optimization for log depth TN circuit

Baseline contraction
  21.403 μs (199 allocations: 10.42 KiB)
opt_einsum contraction
  time for opt_einsum optimization:   0.036493 seconds (17.96 k allocations: 947.227 KiB)
  22.557 μs (209 allocations: 11.48 KiB)
treewidth contraction
  time for treewidth optimization:   0.518001 seconds (1.10 M allocations: 52.986 MiB, 1.22% gc time)
  22.299 μs (209 allocations: 11.48 KiB)

Baseline contraction
  39.232 μs (359 allocations: 19.98 KiB)
opt_einsum contraction
  time for opt_einsum optimization:   0.000511 seconds (25.90 k allocations: 1.594 MiB)
  39.126 μs (352 allocations: 19.27 KiB)
treewidth contraction
  time for treewidth optimization:   0.162353 seconds (450.99 k allocations: 22.284 MiB)
  37.765 μs (362 allocations: 20.33 KiB)

Baseline contraction
  56.189 μs (510 allocations: 29.13 KiB)
opt_einsum contraction
  time for opt_einsum optimization:   0.003387 seconds (168.42 k allocations: 10.358 MiB)
  59.955 μs (513

## Case 2: Tensor Network

Define `rand_local_TN`. This function creates a random tensor network composed of random unitary gates. Note that the ends of this randomly generated tensor networks are 1-Tensors

In [7]:
function rand_local_TN(M, N, max_d, Ngates)

    tensors = Tensor.([[1,0] for i in 1:N])
    contractions = Summation[]
    openidx = [i => 1 for i in 1:N]
    T = TensorNetwork(tensors, contractions, openidx)

    cgc = CircuitGateChain{N}([rand_local_g(M, N, max_d) for i in 1:Ngates])

    tensor_circuit!(T, cgc)
    # measure
    for i in 1:N
        push!(T.tensors, deepcopy(T.tensors[N+1-i]))
        push!(T.contractions, Summation([T.openidx[end], length(T.tensors) => 1]))
        pop!(T.openidx)
    end
    T, cgc
end


rand_local_TN (generic function with 1 method)

In [8]:
printstyled("\nRunning contraction optimization for random TN circuit\n"; color=:green)
for N in 2:4
    Ngates = 10
    M = 2
    max_d = minimum([3, N])
    ψ, cgc = rand_local_TN(M, N, max_d, Ngates)

    printstyled("\nBaseline contraction\n"; color=:yellow)
    ref = contract(ψ)
    printstyled("opt_einsum contraction\n"; color=:yellow)
    einsum_ans = contract(ψ; optimize=true)
    printstyled("treewidth contraction\n"; color=:yellow)
    printstyled("  time for treewidth optimization: "; color=:blue)
    @time optimize_contraction_order!(ψ)
    opt_ans = contract(ψ)

    @test ref ≈ einsum_ans
    @test ref ≈ opt_ans
end



Running contraction optimization for random TN circuit

Baseline contraction
  61.912 μs (562 allocations: 35.00 KiB)
opt_einsum contraction
  time for opt_einsum optimization:   0.009440 seconds (225.14 k allocations: 13.781 MiB, 57.13% gc time)
  63.530 μs (572 allocations: 36.06 KiB)
treewidth contraction
  time for treewidth optimization:   0.003218 seconds (8.42 k allocations: 528.672 KiB)
  66.257 μs (588 allocations: 37.63 KiB)

Baseline contraction
  81.656 μs (729 allocations: 45.53 KiB)
opt_einsum contraction
  time for opt_einsum optimization:   0.024201 seconds (984.14 k allocations: 60.345 MiB, 23.10% gc time)
  80.108 μs (674 allocations: 41.47 KiB)
treewidth contraction
  time for treewidth optimization:   0.000455 seconds (9.04 k allocations: 568.578 KiB)
  81.048 μs (690 allocations: 43.03 KiB)

Baseline contraction
  99.031 μs (877 allocations: 56.64 KiB)
opt_einsum contraction
  time for opt_einsum optimization:   0.037528 seconds (1.40 M allocations: 85.703 MiB, 28

## Case 3: Log Depth MPS

Define `log_depth_mps_TN`. This function creates a random tensor network composed of random unitary gates. Note that these circuits contract to an expectation value (single scalar end value)

In [9]:
function log_depth_mps_TN(M, N, C)
    Nlayers = C*Int(round(log2(N)))
    gates = AbstractCircuitGate{N}[]
    for j in 1:Nlayers
        itergates = (j%2 == 1) ? (1:M:N-M+1) : (M:M:N-M+1)
        for i in itergates
            U = rand_U(M)
            g = CircuitGate{M, N, AbstractGate{M}}(Tuple(collect(i:i+M-1)), MatrixGate(U))
            push!(gates, g)
        end
    end
    cgc = CircuitGateChain{N}(gates)
    # make tensor network

    T = TensorNetwork([], [], [])
    # oldbond = abs(rand(Int, 1)[1] % 10) + 2
    oldbond = 5
    push!(T.tensors, Tensor(randn(ComplexF64, (2, oldbond))))
    for i in 1:N-2
        # newbond = abs(rand(Int, 1)[1] % 10)  + 2
        newbond = 5
        push!(T.tensors, Tensor(randn(ComplexF64, (2, oldbond, newbond))))
        oldbond = newbond
    end
    push!(T.tensors, Tensor(randn(ComplexF64, (2, oldbond))))

    # contract virtual legs
    push!(T.contractions, Summation([1=>2, 2=>2]))
    for i in 2:N-1
        push!(T.contractions, Summation([i=>3, i+1=>2]))
    end

    for i in 1:N
        push!(T.openidx, i=>1)
    end

    tensor_circuit!(T, cgc)
    T_prime = TensorNetwork(copy(T.tensors), copy(T.contractions), copy(T.openidx) )

    l = length(T.tensors)
    step = length(T_prime.tensors)
    for i in 1:l
        push!(T_prime.tensors, conj(T.tensors[i]))
    end

    for i in 1:length(T.openidx)
        push!(T_prime.contractions, Summation([T_prime.openidx[i], shift_pair(T.openidx[i], step)]))
    end

    for (i, con) in enumerate(T.contractions)
        push!(T_prime.contractions, shift_summation(con, step))
    end

    T_prime.openidx=Pair[]

    T, cgc
end

log_depth_mps_TN (generic function with 1 method)

In [10]:
printstyled("\nRunning contraction optimization for log depth MPS circuit\n"; color=:green)
for N in 2:4
    Ngates = 10
    M = 2
    max_d = 3
    ψ, cgc = log_depth_mps_TN(M, N, 1);

    printstyled("\nBaseline contraction\n"; color=:yellow)
    ref = contract(ψ)
    printstyled("opt_einsum contraction\n"; color=:yellow)
    einsum_ans = contract(ψ; optimize=true)
    printstyled("treewidth contraction\n"; color=:yellow)
    printstyled("  time for treewidth optimization: "; color=:blue)
    @time optimize_contraction_order!(ψ)
    opt_ans = contract(ψ)

    @test ref ≈ einsum_ans
    @test ref ≈ opt_ans
end


Running contraction optimization for log depth MPS circuit

Baseline contraction
  11.163 μs (90 allocations: 5.67 KiB)
opt_einsum contraction
  time for opt_einsum optimization:   0.000018 seconds (387 allocations: 23.719 KiB)
  11.172 μs (90 allocations: 5.67 KiB)
treewidth contraction
  time for treewidth optimization:   0.000090 seconds (387 allocations: 23.547 KiB)
  12.384 μs (110 allocations: 8.09 KiB)

Baseline contraction
  25.716 μs (211 allocations: 14.25 KiB)
opt_einsum contraction
  time for opt_einsum optimization:   0.000123 seconds (5.19 k allocations: 323.547 KiB)
  25.632 μs (211 allocations: 13.77 KiB)
treewidth contraction
  time for treewidth optimization:   0.000137 seconds (1.09 k allocations: 67.844 KiB)
  33.671 μs (242 allocations: 32.39 KiB)

Baseline contraction
  40.205 μs (351 allocations: 27.42 KiB)
opt_einsum contraction
  time for opt_einsum optimization:   0.000849 seconds (35.67 k allocations: 2.185 MiB)
  39.608 μs (340 allocations: 25.34 KiB)
treew

## Case 4: Random MPS

Define `log_depth_mps_TN`. This function creates a random tensor network composed of random unitary gates. Note that these circuits contract to an expectation value (single scalar end value)

In [11]:
function rand_local_mps_TN(M, N, max_d, Ngates)

    T = TensorNetwork([], [], [])
    oldbond = abs(rand(Int, 1)[1] % 10) + 2
    push!(T.tensors, Tensor(randn(ComplexF64, (2, oldbond))))
    for i in 1:N-2
        newbond = abs(rand(Int, 1)[1] % 10)  + 2
        push!(T.tensors, Tensor(randn(ComplexF64, (2, oldbond, newbond))))
        oldbond = newbond
    end
    push!(T.tensors, Tensor(randn(ComplexF64, (2, oldbond))))
    # contract virtual legs
    push!(T.contractions, Summation([1=>2, 2=>2]))
    for i in 2:N-1
        push!(T.contractions, Summation([i=>3, i+1=>2]))
    end
    for i in 1:N
        push!(T.openidx, i=>1)
    end

    cgc = CircuitGateChain{N}([rand_local_g(M, N, max_d) for i in 1:Ngates])

    tensor_circuit!(T, cgc)

    T_prime = TensorNetwork(copy(T.tensors), copy(T.contractions), copy(T.openidx) )

    l = length(T.tensors)
    step = length(T_prime.tensors)
    for i in 1:l
        push!(T_prime.tensors, conj(T.tensors[i]))
    end

    for i in 1:length(T.openidx)
        push!(T_prime.contractions, Summation([T_prime.openidx[i], shift_pair(T.openidx[i], step)]))
    end

    for (i, con) in enumerate(T.contractions)
        push!(T_prime.contractions, shift_summation(con, step))
    end

    T_prime.openidx=Pair[]

    T, cgc
end


rand_local_mps_TN (generic function with 1 method)

In [12]:
printstyled("\nRunning contraction optimization for random MPS circuit\n"; color=:green)
for N in 2:4
    Ngates = 10
    M = 2
    max_d = minimum([3, N])
    ψ, cgc = rand_local_mps_TN(M, N, max_d, Ngates);

    printstyled("\nBaseline contraction\n"; color=:yellow)
    ref = contract(ψ)
    printstyled("opt_einsum contraction\n"; color=:yellow)
    einsum_ans = contract(ψ; optimize=true)
    printstyled("treewidth contraction\n"; color=:yellow)
    printstyled("  time for treewidth optimization: "; color=:blue)
    @time optimize_contraction_order!(ψ)
    opt_ans = contract(ψ)

    @test ref ≈ einsum_ans
    @test ref ≈ opt_ans
end


Running contraction optimization for random MPS circuit

Baseline contraction
  53.061 μs (462 allocations: 30.38 KiB)
opt_einsum contraction
  time for opt_einsum optimization:   0.008502 seconds (89.84 k allocations: 5.488 MiB, 78.91% gc time)
  52.926 μs (462 allocations: 30.38 KiB)
treewidth contraction
  time for treewidth optimization:   0.000494 seconds (7.72 k allocations: 485.266 KiB)
  57.798 μs (497 allocations: 34.86 KiB)

Baseline contraction
  68.523 μs (636 allocations: 44.58 KiB)
opt_einsum contraction
  time for opt_einsum optimization:   0.010963 seconds (530.83 k allocations: 32.512 MiB)
  68.284 μs (636 allocations: 43.66 KiB)
treewidth contraction
  time for treewidth optimization:   0.000541 seconds (9.51 k allocations: 632.516 KiB)
  97.772 μs (720 allocations: 68.78 KiB)

Baseline contraction
  77.957 μs (714 allocations: 56.30 KiB)
opt_einsum contraction
  time for opt_einsum optimization:   0.059843 seconds (1.77 M allocations: 108.592 MiB, 31.80% gc time)
  